# Biblioteki

In [ ]:
import requests
from bs4 import BeautifulSoup

# Zadanie 1

## Wysłanie żądania dostępu do zasobu serwera 

In [ ]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-2"

response   = requests.get(url)

In [17]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))

opinions = page_dom.select("div.js_product-review")
print(type(opinions))

opinion = opinions.pop(0)
print(type(opinion))
print(opinion)

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
<div class="user-post user-post__card js_product-review" data-entry-id="15437754">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/7.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
nemesis wawa</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2021-12-28 08:49:18">3 lata temu, </time>
<time datetime="2021-12-17 21:01:22">po 2 tygodniach</time> użytk

## Ekstrakcja składowych pojedynczej opinii 

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation>em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|.review-feature__title--positives~.review-feature__item|pros|
|lista wad|.review-feature__title--negatives~.review-feature__item|cons|
|dla ilu przydatna|[id^="votes-yes"]|useful|
|dla ilu nieprzydatna|[id^="votes-no"]|useless|
|data wystawienia|.user-post__published > time:nth-child(1)["datetime"]|post_date|
|data zakupu|.user-post__published > time:nth-child(2)["datetime"]|purchase_date|

In [76]:
opinion_id = opinion.get("data-entry-id")
author = opinion.select_one("span.user-post__author-name")
recommendation = opinion.select_one("span.user-post__author-recomendation > em")
rating = opinion.select_one("span.user-post__score-count")
content = opinion.select_one("div.user-post__text")
pros = opinion.select(".review-feature__title--positives~.review-feature__item")
cons = opinion.select(".review-feature__title--negatives~.review-feature__item")
useful = opinion.select('[id^="votes-yes"]')
useless = opinion.select('[id^="votes-no"]')
post_date = opinion.select_one(".user-post__published > time:nth-child(1)").get("datetime")
purchase_date = opinion.select_one(".user-post__published > time:nth-child(2)").get("datetime")

# commit -m "data homework HM"

In [87]:
print(f"\nId: {opinion_id}")
print(f"\nAutor: {author.get_text()}")
print(f"\nPoleca? {recommendation.get_text()}")
print(f"\nNa ile? {rating.get_text()}")
print(f"\nOpinia: {content.get_text()}")

print("\nLista zalet:")
for i in pros:
    print(i.get_text())

print("\nLista wad:")
for i in cons:
    print(i.get_text())

print(f"\nDla ilu przydatna? {useful[0].get_text()}")
print(f"\nDla ilu nieprzydatna? {useless[0].get_text()}")
print(f"\nData wystawienia: {post_date}")
print(f"\nData zakupu: {purchase_date}")


Id: 15437754

Autor: 
nemesis wawa

Poleca? Polecam

Na ile? 5/5

Opinia: Zmiana z v8 zdecydowanie na plus. Dużo lżejszy i zwinniejszy. Opcja pokazywania czasu pozostałego do końca pracy baterii bardzo przydatna. Nie trzeba trzymać przycisku włączania.. Solidne wykonanie. Wart swojej ceny :)

Lista zalet:
dużo końcówek
lekki
poręczny
przycisk zasilania
wyświetlacz

Lista wad:
brak

Dla ilu przydatna? 2

Dla ilu nieprzydatna? 0

Data wystawienia: 2021-12-28 08:49:18

Data zakupu: 2021-12-17 21:01:22
